In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import re

### Discount Rate Estimation

In [3]:
zones = ["", "China","emerg","Europe","Global","India","Japan","Rest"]

def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x

def get_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_damodaran/totalbeta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_unleveraged_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_leverage/beta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x

def get_wacc(zone=""):
    path = "data_damodaran/discount_rate_estimation/wacc_damodaran/wacc" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

In [18]:
sp500 = pd.read_excel("data_damodaran/discount_rate_estimation/risk_premium/histimpl.xls")
sp500 = process_df(sp500.iloc[5:,:])
sp500

In [58]:
risk_premium_country = pd.read_excel("data_damodaran/discount_rate_estimation/risk_premium/ctrypremJuly23.xlsx", sheet_name=None)
risk_premium_country = risk_premium_country["ERPs by country"]
loc1 = risk_premium_country[risk_premium_country.iloc[:,0] == "Abu Dhabi"].index.values[0] - 1
loc2 = risk_premium_country[risk_premium_country.iloc[:,0] == "Zambia"].index.values[0] + 1
risk_premium_country = risk_premium_country.iloc[loc1:loc2,:-2]
risk_premium_country = process_df(risk_premium_country)
risk_premium_country

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Country,Africa,Moody's rating,Rating-based Default Spread,Total Equity Risk Premium,Country Risk Premium,"Sovereign CDS, net of US",Total Equity Risk Premium2,Country Risk Premium3
0,Abu Dhabi,Middle East,Aa2,0.005281,0.05749,0.00749,0.0027,0.053829,0.003829
1,Albania,Eastern Europe & Russia,B1,0.048154,0.118298,0.068298,NaN,NaN,NaN
2,Andorra (Principality of),Western Europe,Baa2,0.020368,0.078888,0.028888,NaN,NaN,NaN
3,Angola,Africa,B3,0.069528,0.148613,0.098613,0.1004,0.192399,0.142399
4,Argentina,Central and South America,Ca,0.128369,0.232068,0.182068,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
152,Uruguay,Central and South America,Baa2,0.020368,0.078888,0.028888,0.0052,0.057375,0.007375
153,Uzbekistan,Eastern Europe & Russia,Ba3,0.038473,0.104567,0.054567,NaN,NaN,NaN
154,Venezuela,Central and South America,C,0.175,0.298205,0.248205,0.1114,0.208,0.158
155,Vietnam,Asia,Ba2,0.032187,0.095651,0.045651,0.013,0.068438,0.018438


In [20]:
pd.read_excel("data_damodaran/discount_rate_estimation/treasury_bills/histretSP.xls", sheet_name=None).keys()

### Cash Flow Estimation

In [4]:
def get_capex(zone=""):
    path = "data_damodaran/cash_flow_estimation/CAPEX/capex" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_working_capital(zone=""):
    path = "data_damodaran/cash_flow_estimation/working_capital/wcdata" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_margins(zone=""):
    path = "data_damodaran/cash_flow_estimation/Margin/margin" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_financing_flows(zone=""):
    path = "data_damodaran/cash_flow_estimation/Financing_Flows/finflows" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_goodwill(zone=""):
    path = "data_damodaran/cash_flow_estimation/Goodwill/goodwill" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

## Growth Estimations

In [5]:
def get_growth_ebit(zone=""):
    path = "data_damodaran/growth/growth_ebit/fundgrEB" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_eps(zone=""):
    path = "data_damodaran/growth/growth_eps/fundgr" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_roe(zone=""):
    path = "data_damodaran/growth/ROE/roe" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_historical(zone=""):
    path = "data_damodaran/growth/historical_growth/histgr" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x


In [6]:
def get_multiple_ebitda(zone=""):
    path = "data_damodaran/multiples/ebitda/vebitda" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_multiple_pe(zone=""):
    path = "data_damodaran/multiples/peg/pe" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_multiple_roe(zone=""):
    path = "data_damodaran/multiples/roe/pbv" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_multiple_sales(zone=""):
    path = "data_damodaran/multiples/sales/ps" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

### Return Measures

In [7]:
def get_eva(zone=""):
    path = "data_damodaran/return_measures/EVA" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

In [28]:
usa_eva = get_eva(zones[0])
china_eva = get_eva(zones[1])
emerg_eva = get_eva(zones[2])
europe_eva = get_eva(zones[3])
global_eva = get_eva(zones[4])
india_eva = get_eva(zones[5])
rest_eva = get_eva(zones[6])

In [29]:
x = yf.Ticker("MSFT")
x.get_shares_full(start="2020-01-01", end=None)
x.incomestmt


In [8]:
def drop_identical_columns(df):
    columns_to_drop = []
    for i in range(len(df.columns)):
        for j in range(i+1, len(df.columns)):
            if df.iloc[:, i].equals(df.iloc[:, j]):
                columns_to_drop.append(df.columns[j])
    df.drop(columns=columns_to_drop, inplace=True)
    return df



def columns_for_sql(df):
    column = pd.Series(df.columns).apply(lambda x: str(x).lower().replace(" ","_"))
    column = column.apply(lambda x: re.sub("[^A-Za-z0-9|_]","",x))
    column = column.apply(lambda x: x.replace("__","_"))
    column = list(column)
    return column


                
def table_by_zone(x):
    x = pd.concat([get_betas(x), get_capex(x), get_eva(x), get_financing_flows(x), get_goodwill(x),
               get_growth_ebit(x), get_growth_eps(x), get_growth_historical(x), get_growth_roe(x),
               get_margins(x), get_multiple_ebitda(x), get_multiple_pe(x), get_multiple_roe(x), 
               get_multiple_sales(x), get_taxrates(x), get_unleveraged_betas(x), get_wacc(x), 
               get_working_capital(x)], axis=1)
    x = x.dropna(how="all", axis=1)
    x = x.T.drop_duplicates().T
    x = drop_identical_columns(x)
    x.columns = columns_for_sql(x)
    x.reset_index()
    return x


def discount_rate_by_zone(x):
    x = pd.concat([get_wacc(x), get_betas(x), get_unleveraged_betas(x), get_taxrates(x)], axis=1)
    x = x.T.drop_duplicates().T
    x = drop_identical_columns(x)
    x = x.dropna(how="all", axis=1)
    x.columns = columns_for_sql(x)
    x.reset_index()
    return x


def growth_by_zone(x):
    x = pd.concat([get_growth_ebit(x), get_growth_eps(x), get_growth_historical(x), get_growth_roe(x)], axis=1)
    x = x.T.drop_duplicates().T
    x = drop_identical_columns(x)
    x = x.dropna(how="all", axis=1)
    x.columns = columns_for_sql(x)
    x.reset_index()
    return x


def cash_flow_by_zone():
    x = pd.concat([get_capex(x), get_financing_flows(x), get_goodwill(x), 
                   get_working_capital(x), get_margins], axis=1)
    x = x.T.drop_duplicates().T
    x = drop_identical_columns(x)
    x = x.dropna(how="all", axis=1)
    x.columns = columns_for_sql(x)
    x.reset_index()
    return x

In [34]:
table_by_zone("").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/usa.csv")
table_by_zone("China").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/china.csv")
table_by_zone("Europe").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/europe.csv")
table_by_zone("India").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/india.csv")
table_by_zone("Japan").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/japan.csv")
table_by_zone("Rest").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/CAN-AUS-NZ.csv")
table_by_zone("Global").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/global.csv")

In [35]:
discount_rate_by_zone("").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/usa_discount.csv")
discount_rate_by_zone("China").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/china_discount.csv")
discount_rate_by_zone("Europe").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/europe_discount.csv")
discount_rate_by_zone("India").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/india_discount.csv")
discount_rate_by_zone("Japan").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/japan_discount.csv")
discount_rate_by_zone("Rest").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/CANAUSNZ_discount.csv")
discount_rate_by_zone("Global").to_csv("/Users/leonplaza/Desktop/ironhack/projects/final-project/Database/global_discount.csv")
    

In [40]:
usa = table_by_zone("")

In [47]:
msft = yf.Ticker("AAPL")

0     66288000000.0
1    108949000000.0
2    119437000000.0
3    114301000000.0
Name: EBIT, dtype: object

In [39]:
def pv_fcff(t):
    ebit*(1-tax) + depr[t] - capex[t] + ktn_change[t]

In [54]:
msft.incomestmt#.loc["Reconciled Depreciation",:].sort_index(ascending=True)

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.147192,0.162045,0.133023,0.144282
Normalized EBITDA,125820000000.0,130541000000.0,120233000000.0,77344000000.0
Net Income From Continuing Operation Net Minority Interest,96995000000.0,99803000000.0,94680000000.0,57411000000.0
Reconciled Depreciation,11519000000.0,11104000000.0,11284000000.0,11056000000.0
Reconciled Cost Of Revenue,214137000000.0,223546000000.0,212981000000.0,169559000000.0
EBITDA,125820000000.0,130541000000.0,120233000000.0,77344000000.0
EBIT,114301000000.0,119437000000.0,108949000000.0,66288000000.0
Net Interest Income,NaN,-106000000.0,198000000.0,890000000.0
Interest Expense,NaN,2931000000.0,2645000000.0,2873000000.0


In [53]:
msft.cashflow

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Free Cash Flow,99584000000.0,111443000000.0,92953000000.0,73365000000.0
Repurchase Of Capital Stock,-77550000000.0,-89402000000.0,-85971000000.0,-72358000000.0
Repayment Of Debt,-11151000000.0,-9543000000.0,-8750000000.0,-13592000000.0
Issuance Of Debt,5228000000.0,9420000000.0,20393000000.0,16091000000.0
Issuance Of Capital Stock,NaN,NaN,1105000000.0,880000000.0
Capital Expenditure,-10959000000.0,-10708000000.0,-11085000000.0,-7309000000.0
Interest Paid Supplemental Data,3803000000.0,2865000000.0,2687000000.0,3002000000.0
Income Tax Paid Supplemental Data,18679000000.0,19573000000.0,25385000000.0,9501000000.0
End Cash Position,30737000000.0,24977000000.0,35929000000.0,39789000000.0
Beginning Cash Position,24977000000.0,35929000000.0,39789000000.0,50224000000.0


In [55]:
msft.balancesheet

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Treasury Shares Number,0.0,NaN,NaN,NaN
Ordinary Shares Number,15550061000.0,15943425000.0,16426786000.0,16976763000.0
Share Issued,15550061000.0,15943425000.0,16426786000.0,16976763000.0
Net Debt,81123000000.0,96423000000.0,89779000000.0,74420000000.0
Total Debt,111088000000.0,120069000000.0,124719000000.0,112436000000.0
Tangible Book Value,62146000000.0,50672000000.0,63090000000.0,65339000000.0
Invested Capital,173234000000.0,170741000000.0,187809000000.0,177775000000.0
Working Capital,-1742000000.0,-18577000000.0,9355000000.0,38321000000.0
Net Tangible Assets,62146000000.0,50672000000.0,63090000000.0,65339000000.0
Common Stock Equity,62146000000.0,50672000000.0,63090000000.0,65339000000.0


In [57]:
stock = yf.Ticker("AAPL")
balance_sheet = stock.balance_sheet

# Find short-term interest-bearing debt
if 'ShortLongTermDebt' in balance_sheet.columns:
    short_term_debt = balance_sheet['ShortLongTermDebt']
    print(short_term_debt)
else:
    print("Short-term interest-bearing debt information not available.")

Short-term interest-bearing debt information not available.
